In [17]:
import numpy as np

# To find pgm files
import glob

# To read and show pgm files
from skimage.io import imread, imshow

# Implementations

## Proximal operators

### Soft thresholding operator

In [8]:
def soft_thresholding(mat, epsilon):
    """
    Soft thresholding operator
    Proximal operator of the L1 norm
    :param mat: matrix to threshold
    :param epsilon: value of threshold
    :return mat: thresholded matrix
    """
    mat[np.abs(mat) < epsilon] = 0
    mat[mat > epsilon] -= epsilon
    mat[mat < -epsilon] += epsilon

    return mat

### Singular value thresholding

In [12]:
def svalue_thresholding(mat, epsilon):
    """
    Singular value thresholding
    Proximal operator of the nuclear norm
    :param mat: matrix of svalue to threshold
    :param epsilon: value of threshold
    :return mat: thresholded matrix
    """
    u, s, v = np.linalg.svd(mat, full_matrices = False, compute_uv = 1)
    soft_thresholding(s, epsilon)
    return u.dot(np.diag(s).dot(v))

## Function A=lrmc(X,W,tau,beta)
### Parameters
+ X D × N data matrix.
+ W D × N binary matrix denoting known (1) or missing (0) entries.
+ $\tau$ Parameter of the optimization problem in (3.30).
+ $\beta$ Step size of the dual gradient ascent step in (3.32).

### Returned values
A Low-rank completion of the matrix X.
### Description
Finds a low-rank matrix A whose entries in W coincide with those of X by using the SVT algorithm.

In [32]:
def lrmc(X, W, tau, beta):
    """
    Finds a low-rank matrix A whose entries in W coincide
    with those of X by using the SVT algorithm.
    :param X: DxN data matrix
    :param W: DxN binary matrix denoting known (1) or missing (0) entries
    :param tau: Parameter of the optimization problem
    :param beta: Step size of the dual gradient ascent step
    :return A: Low-rank completion of the mtrix X
    """
    Z = 0
    Z_stop = 1
    while (np.linalg.norm(Z - Z_stop) < 10e-3):
        Z_stop = Z
        A = svalue_thresholding(Z * W, tau)
        Z = Z + beta * (X * W - A * W)
    return A

# Data

In [18]:
IMG_DIR = "images/"
INDIVIDUALS = ["yaleB01", "yaleB02", "yaleB03"]

In [54]:
faces = {}
for k in INDIVIDUALS:
    faces[k] = {}
    faces[k]['files'] = glob.glob(IMG_DIR + k + "/" + k + "_P00A" + "*.pgm")
    faces[k]["data"] = np.concatenate([imread(f).reshape(-1, 1)\
                                       for f in faces[k]['files']], axis = 1)